In [2]:
import re
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
feature_type = '1'

#1 -> just sentence
#2 -> all text

In [5]:
target_type = 1
#1 -> pos

In [6]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [7]:
train.head()

,text,sentence,snp,phenotype,ASSOCIATION,CONFIDENCE
0,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,nicotine metabolism,neutral,zero
1,OBJECTIVE: Maternal smoking during pregnancy i...,We examined whether a common genetic variant a...,rs1051730,fetal growth characteristics,positive,weak
2,OBJECTIVE: Maternal smoking during pregnancy i...,RESULTS: Among mothers who did not smoke durin...,rs1051730,fetal growth characteristic,negative,-
3,OBJECTIVE: Maternal smoking during pregnancy i...,Among mothers who continued smoking during pre...,rs1051730,head circumference,negative,-
4,OBJECTIVE: Maternal smoking during pregnancy i...,The T-allele of maternal rs1051730 was associa...,rs1051730,second and third trimester fetal femur length,positive,weak


In [8]:
train['label'] = train['CONFIDENCE']
test['label']  = test['CONFIDENCE']

In [9]:
train.dropna(subset=['sentence'], inplace=True)
test.dropna(subset=['sentence'], inplace=True)

In [10]:
train = train[train.ASSOCIATION == 'positive']
test  =  test[test.ASSOCIATION  == 'positive']

In [11]:
res = pd.read_excel('res.xlsx')

In [12]:
def vectorize_bow(df_X, cv):
    bow_X = cv.transform(df_X)
    bow_df = pd.DataFrame(data=bow_X.toarray(),
                          columns=[cv.get_feature_names_out()])
    return bow_df

def define_and_run_model(model, X_train, X_test, y_train):
    clf = model
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

def show_result(y_pred, y_test):
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred, digits=3))
    print(accuracy_score(y_test, y_pred))
    
def save_res(model_name, feature_type, target_type, y_pred_sr, y_test_sr, idx):
    res_pre = precision_score(y_test_sr, y_pred_sr, average='weighted')
    res_rec = recall_score(y_test_sr, y_pred_sr, average='weighted')
    res_f1 = f1_score(y_test_sr, y_pred_sr, average='weighted')
    res.at[idx, 'model-name'] = model_name
    res.at[idx, 'feature-type'] = feature_type
    res.at[idx, 'target-type'] = target_type
    res.at[idx, 'accuracy'] = accuracy_score(y_test_sr, y_pred_sr)
    res.at[idx, 'precision'] = res_pre
    res.at[idx, 'recall'] = res_rec
    res.at[idx, 'f1-score'] = res_f1

In [13]:
possible_labels = train.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

train['label'] = train.label.replace(label_dict)
test['label']  = test.label.replace(label_dict)

In [14]:
df = pd.concat([train, test])

In [15]:
bow_vectorizer = CountVectorizer(lowercase=False, 
                                 ngram_range=(1, 1),
                                 max_df=.80,
                                 min_df=1)
bow_vectorizer.fit(df['sentence'])

CountVectorizer(lowercase=False, max_df=0.8)

In [16]:
train_bow_df  = vectorize_bow(list(train['sentence']), bow_vectorizer)
test_bow_df   = vectorize_bow(list(test['sentence']), bow_vectorizer)

In [17]:
X_train = train_bow_df
y_train = train['label']

In [18]:
X_test = test_bow_df
y_test = test['label']

In [19]:
X_train.shape

(701, 2402)

In [20]:
y_train.shape

(701,)

In [21]:
X_test.shape

(170, 2402)

In [22]:
y_test.shape

(170,)

In [23]:
test['label'].value_counts()

0    140
2     16
1     14
Name: label, dtype: int64

In [24]:
%%time
smodel = SVC(kernel='rbf')
smodel.fit(X_train, y_train)
pred_values_svm = smodel.predict(X_test)
show_result(pred_values_svm, y_test)
save_res('svm', feature_type, target_type, pred_values_svm, y_test, len(res))

[[133   7   0]
 [ 10   4   0]
 [ 16   0   0]]
              precision    recall  f1-score   support

           0      0.836     0.950     0.890       140
           1      0.364     0.286     0.320        14
           2      0.000     0.000     0.000        16

    accuracy                          0.806       170
   macro avg      0.400     0.412     0.403       170
weighted avg      0.719     0.806     0.759       170

0.8058823529411765
Wall time: 1.07 s


In [25]:
%%time
lr = LogisticRegression(random_state=0,
                        penalty='l2',
                        solver='newton-cg',
                        C=1.2,
                        max_iter=20)
lr.fit(X_train, y_train)
pred_values_lr = lr.predict(X_test)
show_result(pred_values_lr, y_test)
save_res('LogisticRegression', feature_type, target_type, pred_values_lr, y_test, len(res))

[[117  23   0]
 [  5   9   0]
 [ 14   2   0]]
              precision    recall  f1-score   support

           0      0.860     0.836     0.848       140
           1      0.265     0.643     0.375        14
           2      0.000     0.000     0.000        16

    accuracy                          0.741       170
   macro avg      0.375     0.493     0.408       170
weighted avg      0.730     0.741     0.729       170

0.7411764705882353
Wall time: 475 ms


In [26]:
%%time
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
pred_values_rf = model.predict(X_test)
show_result(pred_values_rf, y_test)
save_res('RandomForest', feature_type, target_type, pred_values_rf, y_test, len(res))

[[140   0   0]
 [  5   9   0]
 [ 11   5   0]]
              precision    recall  f1-score   support

           0      0.897     1.000     0.946       140
           1      0.643     0.643     0.643        14
           2      0.000     0.000     0.000        16

    accuracy                          0.876       170
   macro avg      0.513     0.548     0.530       170
weighted avg      0.792     0.876     0.832       170

0.8764705882352941
Wall time: 1.45 s


In [27]:
%%time
model = KNeighborsClassifier(n_neighbors=2)
model.fit(X_train, y_train)
pred_values_knn = model.predict(X_test)
show_result(pred_values_knn, y_test)
save_res('knn', feature_type, target_type, pred_values_knn, y_test, len(res))

[[123  16   1]
 [ 11   3   0]
 [ 15   1   0]]
              precision    recall  f1-score   support

           0      0.826     0.879     0.851       140
           1      0.150     0.214     0.176        14
           2      0.000     0.000     0.000        16

    accuracy                          0.741       170
   macro avg      0.325     0.364     0.343       170
weighted avg      0.692     0.741     0.716       170

0.7411764705882353
Wall time: 84.7 ms


In [28]:
%%time
model = GaussianNB()
model.fit(X_train, y_train)
pred_values_nb = model.predict(X_test)
show_result(pred_values_nb, y_test)
save_res('GaussianNB', feature_type, target_type, pred_values_nb, y_test, len(res))

[[66 69  5]
 [ 3 10  1]
 [11  4  1]]
              precision    recall  f1-score   support

           0      0.825     0.471     0.600       140
           1      0.120     0.714     0.206        14
           2      0.143     0.062     0.087        16

    accuracy                          0.453       170
   macro avg      0.363     0.416     0.298       170
weighted avg      0.703     0.453     0.519       170

0.45294117647058824
Wall time: 140 ms


In [29]:
%%time
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
pred_values_dt = model.predict(X_test)
show_result(pred_values_dt, y_test)
save_res('DecisionTree', feature_type, target_type, pred_values_dt, y_test, len(res))

[[108  30   2]
 [  4  10   0]
 [  9   6   1]]
              precision    recall  f1-score   support

           0      0.893     0.771     0.828       140
           1      0.217     0.714     0.333        14
           2      0.333     0.062     0.105        16

    accuracy                          0.700       170
   macro avg      0.481     0.516     0.422       170
weighted avg      0.784     0.700     0.719       170

0.7
Wall time: 183 ms


In [30]:
%%time
model = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model.fit(X_train, y_train)
pred_values_gb = model.predict(X_test)
show_result(pred_values_gb, y_test)
save_res('GradientBoosting', feature_type, target_type, pred_values_gb, y_test, len(res))

[[122  15   3]
 [  4   8   2]
 [ 14   1   1]]
              precision    recall  f1-score   support

           0      0.871     0.871     0.871       140
           1      0.333     0.571     0.421        14
           2      0.167     0.062     0.091        16

    accuracy                          0.771       170
   macro avg      0.457     0.502     0.461       170
weighted avg      0.761     0.771     0.761       170

0.7705882352941177
Wall time: 6.44 s


In [ ]:
import os
os.remove("res.xlsx") 

res.to_excel('res.xlsx', index=False)